In [ ]:
!pip install dask[complete]


In [ ]:
import dask.dataframe as dd
import pandas as pd
from huggingface_hub import hf_hub_download
import tqdm as notebook_tqdm

# Исходные файлы загружены на ресурс https://huggingface.co/datasets
REPO_ID = "DmitriiMiptEdu/event_data"
adv = 'adv_data.csv'
event = 'event_data.csv'
users = 'users_data.csv'


In [ ]:
# Импорт данных из CSV файлов
event_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=event, repo_type="dataset"))
adv_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=adv, repo_type="dataset"))
users_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=users, repo_type="dataset"))

In [ ]:
# Преобразование полей датасетов во временные метки
event_data['event_dtm'] = dd.to_datetime(event_data['event_dtm'])
event_data['update_dtm'] = dd.to_datetime(event_data['update_dtm'])
adv_data['campaign_start_dtm'] = dd.to_datetime(adv_data['campaign_start_dtm'])
adv_data['campaign_end_dtm'] = dd.to_datetime(adv_data['campaign_end_dtm'])
users_data['registration_dtm'] = dd.to_datetime(users_data['registration_dtm'])
users_data['delete_dtm'] = dd.to_datetime(users_data['delete_dtm'])


In [ ]:
# Объединение users_data и event_data
merged_data = dd.merge(event_data, users_data, on='user_phone', how='left')

# Объединение с adv_data
merged_data = dd.merge(merged_data, adv_data, on='user_id', how='left')

In [ ]:
# Анализ событий пользователей
# Сортировка событий по пользователям и времени
merged_data = merged_data.sort_values(by=['user_phone', 'event_dtm'])

In [ ]:
# Идентификация конверсий
conversions = merged_data[merged_data['event_status'] == 'success']

In [ ]:
# Разработка модели атрибуции

# Разработка модели атрибуции

In [ ]:
# Реализация модели линейной атрибуции
def linear_attribution(events):
    n = len(events)
    attribution = 1.0 / n
    events['attribution'] = attribution
    return events

# Применение линейной модели атрибуции
attributed_data = merged_data.groupby('user_phone').apply(linear_attribution, meta=merged_data)


In [ ]:
# Визуализация результатов
import matplotlib.pyplot as plt

# Сбор данных для визуализации
source_attribution_pd = source_attribution.compute()

# Пример простой визуализации распределения атрибуции
source_attribution_pd.plot(kind='bar')
plt.title('Атрибуция по источникам трафика')
plt.xlabel('Источник')
plt.ylabel('Атрибуция')
plt.show()